# DATA PREPARATION AND EXPORT

### 1. Import pandas library and read CSV file

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/survey_results_public.csv', sep=",", header=0)
print(len(df))
df.head(5)

### 2. Data Cleaning

#### I. Drop unnecessary columns

In [ ]:
# Q120 - only one unique value - irrelevant information, checking if there more column like this

for column in df:
    unique_values = df[column].unique()
    if len(unique_values) < 5:
        print(f'Column name: {column:20} | unique values: {unique_values}')

# SurveyLentgh, SurveyEase - irrelevant information for further analysis

cols = ["Q120", "SurveyLength", "SurveyEase"]

df.drop(cols, inplace=True, axis=1)

#### II. Remove duplicate and Incomplete Cases

In [ ]:
# Removing rows where more than 60% of column values is null
n_columns = len(df.columns)
# n_incomplete = 0
# for row in df.index:
#     n_nulls = 0
#     for column in df:
#         if pd.isnull(df[column][row]):
#             n_nulls += 1
#     if n_nulls/n_columns > 0.6:
#         n_incomplete += 1
# print(n_incomplete)

# much, much faster way to do so
mostly_nulls = df.isnull().sum(axis=1)
mostly_nulls_count = mostly_nulls[mostly_nulls/n_columns > 0.6]
df.drop(mostly_nulls_count.index, inplace=True)
print(f'Removed {len(mostly_nulls_count)}')

In [ ]:
# Check for duplicates
duplicates = df.iloc[:,1:].duplicated().any()
duplicates

#### III. Threat null values

In [ ]:
cols = [column for column in df if df[column].isnull().sum()>0 and df[column].dtype == 'object']
df = df.fillna(dict.fromkeys(cols, 'NA'))
with pd.option_context('display.max_rows', None):
    display(df.isnull().sum())

#### IV. Handle data types

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df.dtypes)

#### V. Remove nonsense answers

In [ ]:
rows = df[df["CompTotal"] > df["CompTotal"].quantile(0.995)]
df["CompTotal"].quantile(0.995)
display(rows)
df.drop(rows, inplace=True)

In [ ]:
# Years should be integers
df['WorkExp'] = df['WorkExp'].astype('Int32')

In [ ]:
# First approach on CodingActivities
not_null_CA = df['CodingActivities'][df['CodingActivities'].notnull()]

with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(not_null_CA[
        (df['CodingActivities'].str.contains('Hobby') != True) &
        (df['CodingActivities'].str.contains('Freelance/contract') != True) &
        (df['CodingActivities'].str.contains('Contribute to open-source') != True) &
        (df['CodingActivities'].str.contains('Bootstrapping a business') != True) &
        (df['CodingActivities'].str.contains('School or academic') != True) &
        (df['CodingActivities'].str.contains('Professional development') != True) &
        (df['CodingActivities'].str.contains('code outside of work') != True)
        ].head(5))

In [ ]:
# More gentle approach on LearnCode
not_null_LC = df['LearnCode'][df['LearnCode'].notnull()]
listed_LC = not_null_CA.str.split(';')

with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    # Check if there are user inputs and additionally nonsense answers
    display(listed_LC.explode().unique())
    
    # Check if nonsense answer is repeatable
    #display(listed_LC.explode()[listed_CA.explode() == 'answer'])

In [ ]:
# LearnCodeOnline
not_null_LCO = df['LearnCodeOnline'][df['LearnCodeOnline'].notnull()]
listed_LCO = not_null_LCO.str.split(';')
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    # Check if there are user inputs and additionally nonsense answers
    display(listed_LCO.explode().unique())
    
     # Check if nonsense answer is repeatable
    #display(listed_LC.explode()[listed_CA.explode() == 'Click to write Choice 20'])

In [ ]:
# LearnCodeCoursesCert
not_null_LCCC = df['LearnCodeCoursesCert'][df['LearnCodeCoursesCert'].notnull()]
listed_LCCC = not_null_LCCC.str.split(';')
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    # Check if there are user inputs and additionally nonsense answers
    display(listed_LCCC.explode().unique())
    
     # Check if nonsense answer is repeatable
    #display(listed_LCCC.explode()[listed_CA.explode() == 'answers'])

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df["CompTotal"])

In [ ]:

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df['SOAI'])

In [ ]:
size_dict = {}

for column in df:
    if df[column].dtype == "object":
        size_dict[column] = f'string: {df[column].str.len().max()}'
    elif df[column].dtype == 'int64' or df[column].dtype == 'float64':
        size_dict[column] = f'max_number {df[column].dtype}: {df[column].max()}'
    
size_dict



In [ ]:
df["SOAI_len"] = df["SOAI"].str.len()
sorted_soai = df.sort_values(by="SOAI_len", ascending=False)
with pd.option_context("display.max_colwidth", None):
    display(sorted_soai["SOAI"].head(20))

In [ ]:
pd.set_option('display.max_columns', None)
df[df['WorkExp'] == df['WorkExp'].max()].head(5)

In [ ]:
#pd.options.display.float_format = '{:.0f}'.format
#pd.options.display.float_format = '{:.7e}'.format
df['CompTotal'].describe()

In [ ]:
import matplotlib.pyplot as plt

counts, bins = np.histogram(df["CompTotal"])
plt.stairs(counts, bins)

In [ ]:
import numpy as np
with pd.option_context("display.max_columns", None):
    display(df[df['WorkExp'] == df["WorkExp"].isnull()].head(5))

In [ ]:
df["CompTotal"].mean()
